In [1]:
import torch

In [2]:
def predict_globalpointer_compare(y_pre, tags):
    res = [[]] * y_pre.shape[0]
    idx = (y_pre>0).nonzero()
    for b, tag_idx, start, end in idx.tolist():
        res[b].append((tags[tag_idx], (start, end)))
        
    return res


def compare_res(y_pre, y_true, tags, txts, new2ori_idxs):
    y_pre = predict_globalpointer_compare(y_pre, tags)
    y_true = predict_globalpointer_compare(y_true, tags)
    res = []
    for y_pre_cur, y_true_cur, txt, new2ori_idx in zip(y_pre, y_true, txts, new2ori_idxs):
        y_pre_cur, y_true_cur = set(y_pre_cur), set(y_true_cur)
        cur_res = {}
        pre_res, true_res = [], []
        for tag, (start, end) in y_pre_cur - y_true_cur:
            start_txt, end_txt = new2ori_idx[start][0], new2ori_idx[end][1]
            ner_txt = txt[start_txt:end_txt]
            pre_res.append((tag, ner_txt, (start, end), (start_txt, end_txt)))
        for tag, (start, end) in y_true_cur - y_pre_cur:
            start_txt, end_txt = new2ori_idx[start][0], new2ori_idx[end][1]
            ner_txt = txt[start_txt:end_txt]
            true_res.append((tag, ner_txt, (start, end), (start_txt, end_txt)))
        if pre_res:
            cur_res['pre'] = pre_res
        if true_res:
            cur_res['true'] = pre_res
        res.append(cur_res)
    return res

In [3]:
txt = ['非常威锋网服务', '二二多大的大额的我']
new2ori = [[(i, i + 1) for i in range(len(txt[0]))] for _ in range(2)]
tags = {i:l for i,l in enumerate('abcde')}
y_pre = torch.randn(2, len(tags), len(txt[0]), len(txt[0]))
y_true =torch.triu( torch.randint(0, 2, (2, len(tags), len(txt[0]), len(txt[0]))) )

In [4]:
compare_res(y_pre, y_true, tags, txt, new2ori)

[{'pre': [('a', '服务', (5, 6), (5, 7)),
   ('b', '', (5, 0), (5, 1)),
   ('c', '', (3, 0), (3, 1)),
   ('a', '', (5, 4), (5, 5)),
   ('e', '威锋网', (2, 4), (2, 5)),
   ('e', '服务', (5, 6), (5, 7)),
   ('a', '', (5, 2), (5, 3)),
   ('c', '', (3, 2), (3, 3)),
   ('a', '', (5, 0), (5, 1)),
   ('b', '', (5, 4), (5, 5)),
   ('a', '常威锋', (1, 3), (1, 4)),
   ('a', '', (6, 5), (6, 6)),
   ('c', '', (2, 1), (2, 2)),
   ('e', '', (6, 5), (6, 6)),
   ('e', '常', (1, 1), (1, 2)),
   ('b', '', (6, 4), (6, 5)),
   ('a', '', (4, 1), (4, 2)),
   ('b', '', (3, 1), (3, 2)),
   ('a', '', (6, 2), (6, 3)),
   ('a', '', (4, 3), (4, 4)),
   ('a', '非', (0, 0), (0, 1)),
   ('a', '', (3, 1), (3, 2)),
   ('a', '', (3, 0), (3, 1)),
   ('e', '', (6, 0), (6, 1)),
   ('a', '', (3, 2), (3, 3)),
   ('a', '', (5, 1), (5, 2)),
   ('c', '', (2, 0), (2, 1)),
   ('e', '', (3, 1), (3, 2)),
   ('e', '非常威', (0, 2), (0, 3)),
   ('d', '', (3, 2), (3, 3)),
   ('a', '', (6, 1), (6, 2)),
   ('c', '', (6, 5), (6, 6)),
   ('d', '', (3, 0